In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision import datasets, transforms
from pathlib import Path
from tqdm import tqdm
import my_transforms

# Loading data

In [2]:
path_train = './data/train'
path_test = './data/test'

train = Path(path_train)
test = Path(path_test)
print(train)
print(test)

data/train
data/val_good


# Preprocessing data

In [7]:
test_Transform = transforms.Compose([
            transforms.Resize([64, 64]),
            #transforms.RandomRotation(degrees=30, fill=270),
            #transforms.ColorJitter(brightness=0.5, contrast=0.5),
            transforms.ToTensor(),
            transforms.GaussianBlur(kernel_size=3),
            transforms.Normalize(mean = (0.5,0.5,0.5), std = (0.5,0.5,0.5))])

Transform = transforms.Compose([
            transforms.RandomRotation(degrees=30, fill=270),
            transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
            transforms.GaussianBlur(kernel_size=3),
            ])
    
    
train_Transform = transforms.Compose([transforms.Resize([64, 64]),
                                      my_transforms.AddGaussianNoise(0.3), # 0.3為標準差(sigma)
                                      Transform,
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean = (0.5,0.5,0.5), std = (0.5,0.5,0.5))
                                    ])

    
train_data = datasets.ImageFolder(train, transform = train_Transform)
test_data = datasets.ImageFolder(test, transform = test_Transform)

In [8]:
dataset_size = {'train':len(train_data), 'val':len(test_data)}
print(dataset_size)

{'train': 888644, 'val': 218771}


In [9]:
train_data = DataLoader(train_data, batch_size = 130, shuffle = True, num_workers = 4)
test_data = DataLoader(test_data, batch_size = 130, shuffle = False, num_workers = 4)

In [10]:
images, labels = next(iter(train_data))
print(images.shape)
print(labels.shape)

torch.Size([120, 3, 64, 64])
torch.Size([120])


# 裝置設定

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Building model

In [12]:
import torchvision.models as models
densenet161 = models.densenet161(pretrained=False)
densenet161

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [21]:
# 最後一層分類數目從 512 修改成 5716。
num_ftrs = densenet161.classifier.in_features
densenet161.classifier = nn.Linear(num_ftrs, 5716)
model = densenet161.to(device) 

# Optimizer and loss function

In [14]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0005)

In [16]:
from torch.optim import lr_scheduler

decayRate = 0.9999
exp_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

# Training

In [18]:
def train(model, dataloader, dataset_size, criterion, optimizer, scheduler):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    i = 0
    
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        #梯度歸零
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        #backward
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()*inputs.size(0)
        running_corrects += (preds==labels).sum().item()
        
    scheduler.step()
    
    return running_loss/dataset_size, running_corrects/dataset_size

In [19]:
def evaluate(model, dataloader, dataset_size, criterion, k=5):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    top1 = 0
    top2 = 0
    top3 = 0
    top4 = 0
    topk = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            running_loss += loss.item()*inputs.size(0)
            running_corrects += (preds==labels).sum().item()
            
            _,maxk = torch.topk(outputs.data,k,dim=-1)
            total += labels.size(0)
            labels = labels.view(-1,1) 
            top1 += (labels == maxk[:,0:1]).sum().item()
            top2 += (labels == maxk[:,0:2]).sum().item()
            top3 += (labels == maxk[:,0:3]).sum().item()
            top4 += (labels == maxk[:,0:4]).sum().item()
            topk += (labels == maxk[:,0:5]).sum().item()
        print('Accuracy : top1={:.4f}%, top2={:.4f}%, top3={:.4f}%, top4={:.4f}%,top{}={:.4f}%'.format(100*top1/total, 100*top2/total, 100*top3/total, 100*top4/total, k, 100*topk/total))
                
    return running_loss/dataset_size, running_corrects/dataset_size

In [22]:
import time
import copy

num_epochs = 100
dataloader = {'train':train_data, 'test':test_data}

since = time.time()

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs-1}')
    
    train_loss, train_acc = train(model, dataloader['train'], dataset_size['train'], criterion, optimizer, exp_lr_scheduler)
    print(f'loss:{train_loss} acc:{train_acc}')

    test_loss, test_acc = evaluate(model, dataloader['test'], dataset_size['test'], criterion)
    print(f'loss:{test_loss} acc:{test_acc}')

    with open('./output/高斯模型/log.txt', 'a') as f:
        f.write(f'train loss: {train_loss} acc: {train_acc} val loss: {test_loss} acc: {test_acc}')
        f.write('\n')
    
    best_model_wts = copy.deepcopy(model.state_dict())
    model.load_state_dict(best_model_wts)
    torch.save(model, './output/高斯模型/model/model_{}.pth'.format(epoch))

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed//60, time_elapsed%60))

Epoch 0/49


  1%|▊                                                                              | 76/7406 [05:33<8:56:49,  4.39s/it]


KeyboardInterrupt: 